<a href="https://colab.research.google.com/github/duskvirkus/alias-free-gan/blob/devel/notebooks/GPU_Inference_Alias_Free_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Inference - Alias-Free GAN

by duskvirkus

This is a notebook for generating images and latent space walks with Alias-Free GAN on a Colab GPU instance.

Repository: https://github.com/duskvirkus/alias-free-gan

## GPU check

If this fails change the runtime type in Runtime > Change runtime type > Select GPU.

In [1]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-a508f6d5-c4b3-f5fc-ffb7-f635f3a12a41)


## Clone Repository

In [2]:
import os

repo_name = 'alias-free-gan'
git_repo = 'https://github.com/duskvirkus/alias-free-gan.git'
branch_name = 'devel'

working_dir = os.path.join('/content', repo_name)

!git clone --branch {branch_name} {git_repo}
%cd {repo_name}

Cloning into 'alias-free-gan'...
remote: Enumerating objects: 1247, done.
remote: Counting objects: 100% (289/289), done.
remote: Compressing objects: 100% (170/170), done.
remote: Total 1247 (delta 157), reused 194 (delta 102), pack-reused 958
Receiving objects: 100% (1247/1247), 73.53 MiB | 32.30 MiB/s, done.
Resolving deltas: 100% (618/618), done.
/content/alias-free-gan


## Connect Drive

Recomended but not nesscary connect to drive to make accessing models and saving outputs easier. If you don't do this you'll have to upload and download from your colab session.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Dependencies

You will likely see warnings or errors. Ignore these unless you're having trouble in other cells then include the output if you file an issue.

Issue Link: https://github.com/duskvirkus/alias-free-gan/issues/new

In [ ]:
!python install.py

## Set Model and Save Location

In [5]:
model_path = '/content/drive/MyDrive/colab-gpu-alias-free/alias-free-gan/results/training-000005/000145-epoch-checkpoint.pt'
model_size = 256 # 256, 512, or 1024
out_base_path = '/content/drive/MyDrive/alias-free-experiments/testing/training-5-epoch-145/'
import os
os.makedirs(out_base_path, exist_ok=True)

## Setup Output Counter

This is will help avoid overwriting some generated files by accident.

In [6]:
def get_output_count():
  global output_count
  output_count += 1
  return '-' + str(output_count)

output_count = -1

## Generate Single Images

This will generate a folder of images with a seed range. These seeds can be used in the next section for interpolations. Checkout the help for more information on seed values.

In [ ]:
!python scripts/generate_images.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'images' + get_output_count())
!python scripts/generate_images.py \
  --size {model_size} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --seed_stop 9

Using Alias-Free GAN version: 1.0.0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2021-08-03 13:21:01.027564: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 17.3 M
1 | g_ema         | Generator     | 17.3 M
2 | discriminator | Discriminator | 28.9 M
------------------------------------------------
63.5 M    Trainable params
0         Non-trainable params
63.5 M    Total params
253.864   Total estimated model params size (MB)
Loading Model from: /content/drive/MyDrive/colab-gpu-alias-free/alias-free-gan/results/training-000005/000145-epoch-checkpoint.pt

Generating image for seed 0 at truncation 0.750000 and saving to /content/drive/MyDrive/alias-free-experiments/testing/training-5-epoch

## Generate Interpolation

Interpolate been seed values.

**Seeds:**

- If less than two seed values are provided enough will be randomly generated.
- Be sure to start and end at the same seed if you want a loop.

A seeds example
```python
  --seeds 10 1 32 10
```

**Easing:**

You may notice there are a bunch of easing options. Linear is default. All the opitons are python ports of example code from [easings.net](https://easings.net/). Follow the link to view the shape of the different easing methods.

**Notes:**

- As always checkout the help for more information on all available options.
- This automatically compensates for the distance between seed to make animation smoother.

In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'interpolate' + get_output_count())
!python scripts/generate_interpolation.py \
  --size {model_size} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --frames 48

Using Alias-Free GAN version: 1.0.0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2021-08-03 13:23:04.066718: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 17.3 M
1 | g_ema         | Generator     | 17.3 M
2 | discriminator | Discriminator | 28.9 M
------------------------------------------------
63.5 M    Trainable params
0         Non-trainable params
63.5 M    Total params
253.864   Total estimated model params size (MB)
Loading Model from: /content/drive/MyDrive/colab-gpu-alias-free/alias-free-gan/results/training-000005/000145-epoch-checkpoint.pt

Generating 48 frames for interpolation with seeds [11246, 61205] with linear easing.
Saving log of arguments to /content/drive/MyDrive/

## Circular Interpolation

Generates interpolation using sin and cos.

**Diameter:**

Diameter value should be between 0.0 and 10.0. If two are passed, the script will treat them as a min and max and generate a random diameter for each vector dimension.

It's recommended you use low values for low frame counts like 1.5 for 36 frames and much higher values for larger frame counts like 6.0 for 480 frames.

**Notes:**

- Seed values are used diffently from generate images and interpolate.

In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'circular' + get_output_count())
!python scripts/generate_interpolation.py \
  --method circular \
  --size {model_size} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --frames 48 \
  --diameter 2.0

Using Alias-Free GAN version: 1.0.0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2021-08-03 13:38:19.786621: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 17.3 M
1 | g_ema         | Generator     | 17.3 M
2 | discriminator | Discriminator | 28.9 M
------------------------------------------------
63.5 M    Trainable params
0         Non-trainable params
63.5 M    Total params
253.864   Total estimated model params size (MB)
Loading Model from: /content/drive/MyDrive/colab-gpu-alias-free/alias-free-gan/results/training-000005/000145-epoch-checkpoint.pt

Generating 48 frames for circular interpolation with seed 88740.
Saving log of arguments to /content/drive/MyDrive/alias-free-experimen

## Simplex Noise

Generates a interpolation loop useing open simplex noise.

**Diameter:**

No limits on diameter size. Two diameter values will be treated as min and max.

Same thing as above where smaller diameter values for low frame count and higher for more frames.

**Notes:**

- Seed values are used diffently from generate images and interpolate.

In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'simplex_noise' + get_output_count())
!python scripts/generate_interpolation.py \
  --method simplex_noise \
  --size {model_size} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --frames 48 \
  --diameter 0.3

Using Alias-Free GAN version: 1.0.0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2021-08-03 13:45:48.126001: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 17.3 M
1 | g_ema         | Generator     | 17.3 M
2 | discriminator | Discriminator | 28.9 M
------------------------------------------------
63.5 M    Trainable params
0         Non-trainable params
63.5 M    Total params
253.864   Total estimated model params size (MB)
Loading Model from: /content/drive/MyDrive/colab-gpu-alias-free/alias-free-gan/results/training-000005/000145-epoch-checkpoint.pt

Generating 48 frames for simplex noise interpolation with seed 44692.
Saving log of arguments to /content/drive/MyDrive/alias-free-expe

## Repeat / Load Interpolation

By default each interpolation saves a z_vectors.pt file which can be reloaded to repeat the interpolation. As long as the models have the same style dimensions you can use these across different models.

You can also use another method to generate them. The files follow the format of 
```python
torch.save({
  'z_vectors': ## An array of numpy.arrays with shape (1, style_dim).
    ## style_dim is typically 512. (aliasfreeganinstance.generator.style_dim)
    ## Values in these numpy.arrays range between 0 and 1
  })
```

In [ ]:
!python scripts/generate_interpolation.py --help

In [ ]:
out_dir = os.path.join(out_base_path, 'z_vector_load' + get_output_count())
!python scripts/generate_interpolation.py \
  --method load_z_vectors \
  --size {model_size} \
  --load_model {model_path} \
  --outdir {out_dir} \
  --path_to_z_vectors '/content/drive/MyDrive/alias-free-experiments/testing/training-5-epoch-145/simplex_noise-3/z_vectors.pt'

Using Alias-Free GAN version: 1.0.0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
2021-08-03 13:53:01.508180: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 17.3 M
1 | g_ema         | Generator     | 17.3 M
2 | discriminator | Discriminator | 28.9 M
------------------------------------------------
63.5 M    Trainable params
0         Non-trainable params
63.5 M    Total params
253.864   Total estimated model params size (MB)
Loading Model from: /content/drive/MyDrive/colab-gpu-alias-free/alias-free-gan/results/training-000005/000145-epoch-checkpoint.pt

Saving log of arguments to /content/drive/MyDrive/alias-free-experiments/testing/training-5-epoch-145/z_vector_load-4/args_log.txt
Skip

## Rosinality Translate Script

Creates animation with translation on the same latent.

Run the first cell to install latest version of ffmpeg for the video codacs. Required if using `--video True` argument.

In [ ]:
!python scripts/rosinality_generate.py --help

In [8]:
out_dir = os.path.join(out_base_path, 'rosinality_translate' + get_output_count())
!python scripts/rosinality_generate.py \
    --size {model_size} \
    --outdir {out_dir} \
    --n_frame 120 \
    --n_img 1 \
    --frames True \
    {model_path}

Using Alias-Free GAN version: 1.0.0
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 17.3 M
1 | g_ema         | Generator     | 17.3 M
2 | discriminator | Discriminator | 28.9 M
------------------------------------------------
63.5 M    Trainable params
0         Non-trainable params
63.5 M    Total params
253.864   Total estimated model params size (MB)
Loading Model from: /content/drive/MyDrive/colab-gpu-alias-free/alias-free-gan/results/training-000005/000145-epoch-checkpoint.pt

100% 120/120 [20:45<00:00, 10.38s/it]
OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
100% 120/120 [00:00<00:00, 1022.33it/s]
